In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from multiprocessing import Pool

from itertools import product

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [2]:
# Change this according to your directory preferred setting
path_to_train = "./data/train_1"

# This event is in Train_1
event_prefix = "event000001000"

In [3]:
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import DBSCAN, AgglomerativeClustering

class Clusterer(object):
    def __init__(self,rz_scales=[0.65, 0.965, 1.528], eps=0.0035, dz0 = -0.00070, stepdz = 0.00001, stepeps = 0.000005, num_loops=100):                        
        self.rz_scales=rz_scales
        self.epsilon = eps
        self.dz0 = dz0
        self.stepdz = stepdz
        self.stepeps = stepeps
        self.num_loops = num_loops
        
    # remove outliers
    def _eliminate_outliers(self,labels,M):
        norms=np.zeros((len(labels)),np.float32)
        indices=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            index = np.reshape(index,(index.shape[0]))
            indices[i] = len(index)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold1 = np.percentile(norms,90)*5
        threshold2 = 25
        threshold3 = 6
        for i, cluster in enumerate(labels):
            if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
                self.clusters[self.clusters==cluster]=0   
    
    # not sure what this function does?
    def _test_quadric(self,x):
        if x.size == 0 or len(x.shape)<2:
            return 0
        Z = np.zeros((x.shape[0],10), np.float32)
        Z[:,0] = x[:,0]**2
        Z[:,1] = 2*x[:,0]*x[:,1]
        Z[:,2] = 2*x[:,0]*x[:,2]
        Z[:,3] = 2*x[:,0]
        Z[:,4] = x[:,1]**2
        Z[:,5] = 2*x[:,1]*x[:,2]
        Z[:,6] = 2*x[:,1]
        Z[:,7] = x[:,2]**2
        Z[:,8] = 2*x[:,2]
        Z[:,9] = 1
        v, s, t = np.linalg.svd(Z,full_matrices=False)        
        smallest_index = np.argmin(np.array(s))
        T = np.array(t)
        T = T[smallest_index,:]        
        norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
        return norm

    # standard scale our data
    def _preprocess(self, hits):
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        hits['x2'] = x/r
        hits['y2'] = y/r

        r = np.sqrt(x**2 + y**2)
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
        for i, rz_scale in enumerate(self.rz_scales):
            X[:,i] = X[:,i] * rz_scale
       
        return X
    
    def _init(self,dfh):
        # create the radius of the hit from the coordinates
        dfh['r'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2+dfh['z'].values**2)
        
        # radius excluding z coordinate
        dfh['rt'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2)
        
        # create some other features
        dfh['a0'] = np.arctan2(dfh['y'].values,dfh['x'].values)
        dfh['z1'] = dfh['z'].values/dfh['rt'].values
        dfh['x2'] = 1/dfh['z1'].values
        dfh['z2'] = dfh['z']/dfh['r']
        
        # step params
        dz0 = self.dz0
        stepdz = self.stepdz
        stepeps = self.stepeps
        mm = 1
        
        # loop from 1 to 100
#         for ii in tqdm(range(100)):
        for ii in range(self.num_loops):
            mm = mm*(-1)
            dz = mm*(dz0+ii*stepdz)
            # below line copied from r script with better results than ours
            dfh['a1'] = dfh['a0'].values+mm*(dfh['rt']+0.000005*dfh['rt']**2)/1000*(ii/2)/180*math.pi
#             dfh['a1'] = dfh['a0'].values+dz*dfh['z'].values*np.sign(dfh['z'].values)
            dfh['sina1'] = np.sin(dfh['a1'].values)
            dfh['cosa1'] = np.cos(dfh['a1'].values)
            dfh['x1'] = dfh['a1'].values/dfh['z1'].values
            ss = StandardScaler()
            dfs = ss.fit_transform(dfh[['sina1','cosa1','z1','z2','x1','x2']].values)
            cx = np.array([1.2, 1.2, 0.49, 0.28, 0.5, 0.5])
            dfs = np.multiply(dfs, cx)
            
            # cluster our hits
            clusters=DBSCAN(eps=self.epsilon+ii*stepeps,min_samples=1,metric='euclidean',n_jobs=1).fit(dfs).labels_            
    
            # if we are in first iteration add clusters to our hits
            if ii==0:
                dfh['s1'] = clusters
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
            
            # else update our hits conditionally, if it's a better fit
            else:
                # put our new clusters to another feature
                dfh['s2'] = clusters
                
                # get the count of those clusters
                dfh['N2'] = dfh.groupby('s2')['s2'].transform('count')
                maxs1 = dfh['s1'].max()
                
                # if our new clusters are bigger, but less than 20, use the new ones instead
                cond = np.where((dfh['N2'].values>dfh['N1'].values) & (dfh['N2'].values<20))
                s1 = dfh['s1'].values
                s1[cond] = dfh['s2'].values[cond]+maxs1
                
                # write the new clusters back to our dataframe
                dfh['s1'] = s1
                dfh['s1'] = dfh['s1'].astype('int64')
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
        
        # return our clusters
        return dfh['s1'].values    
    
    def predict(self, hits):    
        # init our clusters
        self.clusters = self._init(hits) 
        
        # preprocess our data
        X = self._preprocess(hits) 
        
        # create our last clusterer
        cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7, metric='braycurtis',cluster_selection_method='leaf',algorithm='best', leaf_size=50)
            
        # labels = unique clusters
        labels = np.unique(self.clusters)
        
        # remove outliers
        self._eliminate_outliers(labels,X)
        
        # init n_labels
        n_labels = 0
        
        # now we loop through the points that haven't been assigned to a cluster and assign them with
        # HDBSCAN
        while n_labels < len(labels):
            n_labels = len(labels)
            max_len = np.max(self.clusters)
            mask = self.clusters == 0
            self.clusters[mask] = cl.fit_predict(X[mask])+max_len
            
        return self.clusters

In [4]:
results_df = pd.read_pickle("gs_results.pkl")
# results_df.sort_values("acc", ascending=False)

In [13]:
# defaults: 
# eps=0.0035, dz0 = -0.00070, stepdz = 0.00001, stepeps = 0.000005

# create our params to iterate over
eps_vals = [0.00375, 0.0037]
dz0_vals = [-0.00072, -0.00075, -0.00070,]
stepdz_vals = [0.000010, ]
stepeps_vals = [0.000005]
loop_vals = [110, 120, 125]

foo = product(eps_vals, dz0_vals, stepdz_vals, stepeps_vals, loop_vals)

offset = np.max(results_df.index.values) + 1

iter_list = []
for i, item in enumerate(foo):
    iter_list.append([i + offset, item])

In [14]:
len(iter_list)

18

In [15]:
def grid_search_loop(params):
    counter = params[0]
    eps, dz0, stepdz, stepeps, loops = params[1]
    dataset_scores = []
    print("Evaluating Counter:", counter, "Eps:", eps, "dz0:", dz0, "stepdz:", stepdz, "stepeps:", stepeps, "loops:", loops)
    
    for event_id, hits, cells, particles, truth in load_dataset("./data/train_1", skip=50, nevents=10):
        # Track pattern recognition
        model = Clusterer(eps=eps, dz0=dz0, stepdz=stepdz, stepeps=stepeps, num_loops=loops)
        labels = model.predict(hits)

        # Prepare submission for an event
        one_submission = create_one_event_submission(event_id, hits, labels)

        # Score for the event
        score = score_event(truth, one_submission)
        dataset_scores.append(score)
        
    mean_score = np.mean(dataset_scores)
    
    # create and return our results
    result = [counter, [eps, dz0, stepdz, stepeps, loops, mean_score]]
    print(result)
    
    return result

def grid_search(iter_list, results_df=None, start=0, end=5):
    pool = Pool(processes=14)
    results = pool.map(grid_search_loop, iter_list[start:end])
    pool.close()
    
    return results

In [ ]:
results = grid_search(iter_list, results_df, start=0, end=28)

for item in results:
#     results_df.append(pd.Series(item[1], index=results_df.columns), ignore_index=True)
    results_df.loc[item[0]] = item[1]
    
results_df.sort_values("acc", ascending=False).to_pickle("gs_results.pkl")
results_df

Evaluating Counter: 381 Eps: 0.00375 dz0: -0.00072 stepdz: 1e-05 stepeps: 5e-06 loops: 125
Evaluating Counter: 380 Eps: 0.00375 dz0: -0.00072 stepdz: 1e-05 stepeps: 5e-06 loops: 120
Evaluating Counter: 384 Eps: 0.00375 dz0: -0.00075 stepdz: 1e-05 stepeps: 5e-06 loops: 125
Evaluating Counter: 379 Eps: 0.00375 dz0: -0.00072 stepdz: 1e-05 stepeps: 5e-06 loops: 110
Evaluating Counter: 382 Eps: 0.00375 dz0: -0.00075 stepdz: 1e-05 stepeps: 5e-06 loops: 110
Evaluating Counter: 383 Eps: 0.00375 dz0: -0.00075 stepdz: 1e-05 stepeps: 5e-06 loops: 120
Evaluating Counter: 386 Eps: 0.00375 dz0: -0.0007 stepdz: 1e-05 stepeps: 5e-06 loops: 120
Evaluating Counter: 385 Eps: 0.00375 dz0: -0.0007 stepdz: 1e-05 stepeps: 5e-06 loops: 110
Evaluating Counter: 387 Eps: 0.00375 dz0: -0.0007 stepdz: 1e-05 stepeps: 5e-06 loops: 125
Evaluating Counter: 388 Eps: 0.0037 dz0: -0.00072 stepdz: 1e-05 stepeps: 5e-06 loops: 110
Evaluating Counter: 390 Eps: 0.0037 dz0: -0.00072 stepdz: 1e-05 stepeps: 5e-06 loops: 125
Eval

/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


...

/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


/home/skooch/.local/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


In [11]:
results_df.sort_values("acc", ascending=False)

,eps,dz0,stepdz,stepeps,loops,acc
357,0.00375,-0.00075,0.00001,0.000005,120.0,0.459203
375,0.00370,-0.00075,0.00001,0.000005,120.0,0.459084
356,0.00375,-0.00075,0.00001,0.000005,110.0,0.458654
374,0.00370,-0.00075,0.00001,0.000005,110.0,0.458431
354,0.00375,-0.00072,0.00001,0.000005,120.0,0.458368
353,0.00375,-0.00072,0.00001,0.000005,110.0,0.458337
355,0.00375,-0.00075,0.00001,0.000005,105.0,0.457638
372,0.00370,-0.00072,0.00001,0.000005,120.0,0.457637
359,0.00375,-0.00070,0.00001,0.000005,110.0,0.457582
373,0.00370,-0.00075,0.00001,0.000005,105.0,0.457578


In [12]:
results_df.sort_values("acc", ascending=False).to_pickle("gs_results.pkl")